In [129]:
import pandas as pd
pd.set_option('display.max_columns', None) 
from os import listdir
import os
import glob
import spacy
from collections import Counter
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nlp

[nltk_data] Downloading package stopwords to /Users/nyzy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [130]:
#load data
job_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/data"
courses_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/courses_data"
skills_path = "/Users/nyzy/Library/CloudStorage/GoogleDrive-nitinmali999@gmail.com/.shortcut-targets-by-id/1yEbTjc1DwwTDd2CB86Il6F-3DhFUn8pU/Data Mining/Datasets/skills_df_updated.csv"

files_j= [f for f in listdir(job_path) if f.endswith(".csv")]
df_j = pd.concat([pd.read_csv(os.path.join(job_path,f)) for f in files_j])

files_c= [f for f in listdir(courses_path) if f.endswith(".csv")]
df_c = pd.concat([pd.read_csv(os.path.join(courses_path,f)) for f in files_c])

#files_s= [f for f in listdir(skills_path) if f.endswith(".csv")]
df_s = pd.read_csv(skills_path)



In [3]:
len(df_s["Unnamed: 5"].unique())

904

In [131]:


# Ensure you have the necessary NLTK tokens downloaded
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

def preprocess_text_spacy(text):
    # Use spaCy's pipeline for processing the passed text
    doc = nlp(text)
    preprocessed_text= []
    # Return a preprocessed version of the text (for instance, lemmatized, lowercased, no stop words/punctuations)
    for token in doc:
        # Skip stop words and punctuation
        if token.is_stop or token.is_punct or token.is_space:
            continue
        # Special case for 'data' to keep it in plural form
        if token.lemma_ == 'datum':
            preprocessed_text.append('data')
        else:
            preprocessed_text.append(token.text.lower())

    return ' '.join(preprocessed_text)


# Function to clean and lemmatize job title text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)





[nltk_data] Downloading package punkt to /Users/nyzy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/nyzy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/nyzy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [132]:
#clean skills dataset 

skills = pd.DataFrame(df_s[df_s["Unnamed: 5"].notna()]["Unnamed: 5"].unique())
skills.columns=["skills"]

In [133]:
skills["skills"] = skills["skills"].apply(preprocess_text_spacy)
skills["skills"] = skills["skills"].apply(clean_text)

In [134]:
skills["skills"].tolist()

['visualization',
 'b testing',
 'ansi c',
 'api gateway',
 'api management',
 'apl programming language',
 'asp net',
 'asp net core',
 'asp net core mvc',
 'asp net extension ajax',
 'asp net fundamental',
 'asp net identity',
 'asp net mvc',
 'asp net mvc framework',
 'asp net razor',
 'asp net web api',
 'awk programming language',
 'aws amplify',
 'aws app mesh',
 'aws appsync',
 'aws auto scaling',
 'aws backup',
 'aws batch',
 'aws cli command line interface',
 'aws cloud development kit cdk',
 'aws cloudformation',
 'aws cloudhsm',
 'aws cloudtrail',
 'aws codebuild',
 'aws codecommit',
 'aws codedeploy',
 'aws codepipeline',
 'aws cost management',
 'aws directory service',
 'aws elastic beanstalk',
 'aws elastic mapreduce emr',
 'aws fargate',
 'aws glue',
 'aws identity access management iam',
 'aws inferentia',
 'aws internet thing iot',
 'aws key management service km',
 'aws kinesis',
 'aws lambda',
 'aws opsworks',
 'aws outpost',
 'aws sdk',
 'aws sagemaker',
 'aws serv

In [135]:
def clean_lem_stop(df,column_name):
    df[column_name] = df[column_name].apply(preprocess_text_spacy)
    df[column_name] = df[column_name].apply(clean_text)
    return df
df_j = df_j[:100]
df_c = df_c[:100]
df_j = clean_lem_stop(df_j,"title")
df_j = clean_lem_stop(df_j,"jobDescription")

In [136]:
job_descriptions = df_j["jobDescription"].tolist()
skills_list = skills["skills"].tolist()

df_c["full_description"] = df_c["description"]+df_c["what_you_will_learn_data"]+df_c["objectives"]
df_j = clean_lem_stop(df_c,"full_description")
course_description = df_c["full_description"].tolist()

job_descriptions[1]

'direct hire opportunity client hybrid role located minneapolis mn candidate able work sponsorship seeking highly motivated detail oriented data engineer join dynamic team ideal candidate responsible developing maintaining data solution drive business objective forward passion working data technical background desire contribute project leverage advanced analytics role excellent opportunity meaningful impact work collaboratively team create innovative data driven solution focus data engineering cloud technology data analytics product advanced analytics ready career level difference data driven world encourage apply responsibility data solution development design develop implement data solution address business need leveraging understanding data modeling sql proficiency coding skill language python c javascript continuous improvement identify seize opportunity enhance existing data solution optimizing performance scalability efficiency data visualization collaborate cross functional team

1518

In [118]:
# parse job description to identify required skills from skill list 


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Let's say you have your job descriptions, course descriptions, and skills
job_descriptions = df_j["jobDescription"].tolist() # List of job description texts
#course_descriptions = filtered_keywords_c # List of course description texts
skills_list = skills["skills"].tolist()
  # List of skills
# Combine job descriptions and course descriptions
all_descriptions = job_descriptions 

# Vectorize the descriptions and skills
tfidf_vectorizer = TfidfVectorizer()
all_descriptions_tfidf = tfidf_vectorizer.fit_transform(all_descriptions)

# Transform skills with the same vectorizer
skills_tfidf = tfidf_vectorizer.transform(skills_list)

# Calculate cosine similarity and rank skills
def rank_skills_for_description(description_tfidf, skills_tfidf):
    cosine_similarities = cosine_similarity(description_tfidf, skills_tfidf).flatten()
    
    # Get the top matching skills indices and scores
    top_skills_indices = cosine_similarities.argsort()[::-1]
    
    # Map indices to skill names and scores
    top_skills = [(skills_list[i], cosine_similarities[i]) for i in top_skills_indices]

    return top_skills

# Example: Get top skills for the first job description
top_skills_for_first_job = rank_skills_for_description(all_descriptions_tfidf[1], skills_tfidf)

# Now, to avoid printing out all skills, we slice the list to get the top N
N = 100
print(top_skills_for_first_job[:200])


[('data partitioning', 0.36450003161067684), ('data entry', 0.36450003161067684), ('data transmission', 0.36450003161067684), ('data transport utility', 0.36450003161067684), ('data transposition', 0.36450003161067684), ('data vault', 0.36450003161067684), ('data monetization', 0.36450003161067684), ('data masking', 0.36450003161067684), ('data localization', 0.36450003161067684), ('data hiding encapsulation', 0.36450003161067684), ('data grid', 0.36450003161067684), ('data farming', 0.36450003161067684), ('data explorer', 0.36450003161067684), ('data ethic', 0.36450003161067684), ('data erasure', 0.36450003161067684), ('data encryption', 0.36450003161067684), ('data synchronization', 0.36450003161067684), ('data encoding', 0.36450003161067684), ('data dictionary', 0.36450003161067684), ('data corruption', 0.36450003161067684), ('data conditioning', 0.36450003161067684), ('data compression', 0.36450003161067684), ('data comparison', 0.36450003161067684), ('data caching', 0.364500031610

In [119]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# Initialize the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to get BERT embeddings
def get_bert_embeddings(texts):
    # Tokenize and encode sequences in the list of textual data
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform mean pooling to get sentence-level embeddings
    input_mask_expanded = encoded_input['attention_mask'].unsqueeze(-1).expand(model_output.last_hidden_state.size()).float()
    sum_embeddings = torch.sum(model_output.last_hidden_state * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

# Let's say you have your job descriptions, course descriptions, and skills in DataFrames

# Combine job descriptions and course descriptions
all_descriptions = job_descriptions 

# Get BERT embeddings for descriptions and skills
all_descriptions_embeddings = get_bert_embeddings(all_descriptions)
skills_embeddings = get_bert_embeddings(skills_list)

# Convert embeddings to numpy arrays for cosine similarity
all_descriptions_embeddings_np = all_descriptions_embeddings.numpy()
skills_embeddings_np = skills_embeddings.numpy()

# Calculate cosine similarity and rank skills
def rank_skills_for_description(description_embedding, skills_embeddings):
    cosine_similarities = cosine_similarity([description_embedding], skills_embeddings).flatten()
    
    # Get the top matching skills indices and scores
    top_skills_indices = cosine_similarities.argsort()[::-1]
    
    # Map indices to skill names and scores
    top_skills = [(skills_list[i], cosine_similarities[i]) for i in top_skills_indices]

    return top_skills

# Example: Get top skills for the first job description
top_skills_for_first_job = rank_skills_for_description(all_descriptions_embeddings_np[1], skills_embeddings_np)

# Now, to avoid printing out all skills, we slice the list to get the top N
N = 100
print(top_skills_for_first_job[:N])


[('data warehousing business intelligence dwbi', 0.538889), ('architect engineer contract administration support system', 0.52003807), ('data warehouse appliance', 0.4981823), ('tableau business intelligence software', 0.4980299), ('emulator high level language application program interface ehllapi', 0.49613374), ('visualization toolkit vtk', 0.4955115), ('data ontap server appliance', 0.49481046), ('decision support operation maintenance', 0.49081674), ('business analysis body knowledge babok', 0.48497632), ('cross functional project management', 0.48326913), ('amazon simple workflow service swf', 0.48174405), ('cloud infrastructure management interface cimi', 0.4807723), ('application lifecycle management alm software', 0.47936153), ('data center infrastructure management cim', 0.47277617), ('webspeed openedge advanced business language', 0.47124884), ('data analysis display dadisp', 0.4686407), ('user interface ui design', 0.46846822), ('data access object dao pattern', 0.46842873),

In [120]:
job_descriptions[1]

'direct hire opportunity client hybrid role located minneapolis mn candidate able work sponsorship seeking highly motivated detail oriented data engineer join dynamic team ideal candidate responsible developing maintaining data solution drive business objective forward passion working data technical background desire contribute project leverage advanced analytics role excellent opportunity meaningful impact work collaboratively team create innovative data driven solution focus data engineering cloud technology data analytics product advanced analytics ready career level difference data driven world encourage apply responsibility data solution development design develop implement data solution address business need leveraging understanding data modeling sql proficiency coding skill language python c javascript continuous improvement identify seize opportunity enhance existing data solution optimizing performance scalability efficiency data visualization collaborate cross functional team

,Unnamed: 0,id,title,url,price_detail,is_paid,visible_instructors,locale,description,headline,created,num_subscribers,discount,discount_price,rating,num_reviews,num_quizzes,num_lectures,num_curriculum_items,features,image,primary_category,primary_subcategory,requirements_data,what_you_will_learn_data,labels,target_audiences,estimated_content_length,content_info,instructional_level,objectives,has_certificate
0,0,950390,"Machine Learning A-Z™: AI, Python & R + ChatGP...",https://www.udemy.com/course/machinelearning/,"{'amount': 109.99, 'currency': 'USD', 'price_s...",True,"[{'title': 'Kirill Eremenko', 'name': 'Kirill'...","{'locale': 'en_US', 'title': 'English (US)', '...",<p>Interested in the field of Machine Learning...,Learn to create Machine Learning Algorithms in...,2016-09-05T09:54:22Z,993456,NaN,NaN,4.513425,176950,39,468,507,"{'discussions_create': True, 'discussions_view...",https://img-c.udemycdn.com/course/750x422/9503...,"{'id': 288, 'title': 'Development', 'title_cle...","{'id': 558, 'title': 'Data Science', 'title_cl...",['Just some high school mathematics level.'],"['Master Machine Learning on Python & R', 'Hav...","[{'id': 5336, 'title': 'Data Science', 'url': ...","['Anyone interested in Machine Learning.', 'St...",2559,42.5 total hours,All Levels,"['Master Machine Learning on Python & R', 'Hav...",True
1,1,903744,Python for Data Science and Machine Learning B...,https://www.udemy.com/course/python-for-data-s...,"{'amount': 99.99, 'currency': 'USD', 'price_st...",True,"[{'title': 'Jose Portilla', 'name': 'Jose', 'd...","{'locale': 'en_US', 'title': 'English (US)', '...",<p>Are you ready to start your path to becomin...,"Learn how to use NumPy, Pandas, Seaborn , Matp...",2016-07-13T05:22:58Z,674125,NaN,NaN,4.588927,136746,1,184,185,"{'discussions_create': True, 'discussions_view...",https://img-c.udemycdn.com/course/750x422/9037...,"{'id': 288, 'title': 'Development', 'title_cle...","{'id': 558, 'title': 'Data Science', 'title_cl...","['Some programming experience', 'Admin permiss...",['Use Python for Data Science and Machine Lear...,"[{'id': 5336, 'title': 'Data Science', 'url': ...",['This course is meant for people with at leas...,1494,25 total hours,All Levels,['Use Python for Data Science and Machine Lear...,True
2,2,1754098,The Data Science Course: Complete Data Science...,https://www.udemy.com/course/the-data-science-...,"{'amount': 109.99, 'currency': 'USD', 'price_s...",True,"[{'title': '365 Careers', 'name': '365', 'disp...","{'locale': 'en_US', 'title': 'English (US)', '...",<p><strong>The Problem</strong></p><p>Data sci...,"Complete Data Science Training: Mathematics, S...",2018-06-18T12:30:08Z,634583,NaN,NaN,4.594564,131509,281,518,799,"{'discussions_create': True, 'discussions_view...",https://img-c.udemycdn.com/course/750x422/1754...,"{'id': 288, 'title': 'Development', 'title_cle...","{'id': 558, 'title': 'Data Science', 'title_cl...",['No prior experience is required. We will sta...,['The course provides the entire toolbox you n...,"[{'id': 5336, 'title': 'Data Science', 'url': ...",['You should take this course if you want to b...,1906,32 total hours,All Levels,['The course provides the entire toolbox you n...,True
3,3,765242,R Programming A-Z™: R For Data Science With Re...,https://www.udemy.com/course/r-programming/,"{'amount': 94.99, 'currency': 'USD', 'price_st...",True,"[{'title': 'Kirill Eremenko', 'name': 'Kirill'...","{'locale': 'en_US', 'title': 'English (US)', '...",<p>Learn R Programming by doing!</p><p>There a...,Learn Programming In R And R Studio. Data Anal...,2016-02-17T09:34:04Z,261248,NaN,NaN,4.662838,51993,5,89,94,"{'discussions_create': True, 'discussions_view...",https://img-b.udemycdn.com/course/750x422/7652...,"{'id': 288, 'title': 'Development', 'title_cle...","{'id': 558, 'title': 'Data Science', 'title_cl...",['No prior knowledge or experience needed. Onl...,"['Learn to program in R at a good level', 'Lea...","[{'id': 5332, 'title': 'Data Mining', 'url': '...",['This co

In [86]:
df_j["jobDescription"][0]

'randstad technology data center site engineer located wilmington de relocation assistance available global financial leader exciting data center opportunity physical hand essential worker creative schedule enables extra day compared normal day week family free weekly covid testing opportunity growth advancement unique workload day thing day world class data center seeking experienced data center technician long term project role located wilmington de area basic responsibility include limited rack stack enterprise class technology network storage enterprise server running tracing dressing testing copper fiber cable patch panel device familiarity patch panel troubleshooting diagnostics hardware connectivity commission decommission hardware able perform component replacement dimms hard drive power supply adhere change control process required skill previous experience working complex high availability data center environment providing layer hardware installation ongoing maintenance probl

In [87]:
job_descriptions[0]

'randstad technology data center site engineer located wilmington de relocation assistance available global financial leader exciting data center opportunity physical hand essential worker creative schedule enables extra day compared normal day week family free weekly covid testing opportunity growth advancement unique workload day thing day world class data center seeking experienced data center technician long term project role located wilmington de area basic responsibility include limited rack stack enterprise class technology network storage enterprise server running tracing dressing testing copper fiber cable patch panel device familiarity patch panel troubleshooting diagnostics hardware connectivity commission decommission hardware able perform component replacement dimms hard drive power supply adhere change control process required skill previous experience working complex high availability data center environment providing layer hardware installation ongoing maintenance probl

In [95]:
'''
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Your job description
text = randstad technology data center site engineer located wilmington de relocation assistance available global financial leader exciting data center opportunity physical hand essential worker creative schedule enables extra day compared normal day week family free weekly covid testing opportunity growth advancement unique workload day thing day world class data center seeking experienced data center technician long term project role located wilmington de area basic responsibility include limited rack stack enterprise class technology network storage enterprise server running tracing dressing testing copper fiber cable patch panel device familiarity patch panel troubleshooting diagnostics hardware connectivity commission decommission hardware able perform component replacement dimms hard drive power supply adhere change control process required skill previous experience working complex high availability data center environment providing layer hardware installation ongoing maintenance problem identification incident resolution strong working knowledge data center process design hand expertise midrange system large scale server environment ibm hp oracle amp dell server hardware exceptional troubleshooting problem resolution skill ability document complex problem resolution summary repetitive task interface online ticketing system understanding switch port configuration critical thinking decision making related outage risk assessment strong cable management skill experience managing fiber amp copper data center environment desired skill certification server network technical writing documentation skill preferred proficient microsoft office role operates hour shift amp rotating schedule benefit condition waiting period apply duration long term long term opportunity growth professional development promote job type time pay hour benefit k dental insurance employee assistance program employee discount health insurance life insurance paid time referral program vision insurance schedule hour shift overtime weekend needed covid consideration regular free testing education high school equivalent preferred experience data center year preferred work location person

# Candidate labels
candidate_labels = skills_list

# Using the classifier to predict the relevance of each candidate label to the text
results = classifier(text, candidate_labels)
print(results)
'''

'\nfrom transformers import pipeline\n\nclassifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")\n\n# Your job description\ntext = randstad technology data center site engineer located wilmington de relocation assistance available global financial leader exciting data center opportunity physical hand essential worker creative schedule enables extra day compared normal day week family free weekly covid testing opportunity growth advancement unique workload day thing day world class data center seeking experienced data center technician long term project role located wilmington de area basic responsibility include limited rack stack enterprise class technology network storage enterprise server running tracing dressing testing copper fiber cable patch panel device familiarity patch panel troubleshooting diagnostics hardware connectivity commission decommission hardware able perform component replacement dimms hard drive power supply adhere change control process

In [141]:
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.util import filter_spans

nlp = spacy.load("en_core_web_sm")  # Load the model
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")  # Create the matcher object

# Assuming 'skills_list' is a list of skills, and 'job_descriptions' is a list containing job descriptions

# Add patterns to the matcher. Patterns are made by converting each skill string into a Doc object
patterns = [nlp.make_doc(skill) for skill in skills_list]
matcher.add("Skills", patterns)

# Process the job description to create a Spacy Doc
doc = nlp(job_descriptions[4])

# Match the patterns to the doc
matches = matcher(doc)

# Create Span objects for the matched sequences
spans = [Span(doc, start, end, label="SKILL") for match_id, start, end in matches]

# Filter the spans to remove overlaps
filtered_spans = filter_spans(spans)

# Now you can create new entities in the doc using the filtered spans
doc.ents = filtered_spans  # Overwrite or append to doc.ents with the non-overlapping skill entities

# Print the entities in the document
for ent in doc.ents:
    print(ent.text, ent.label_)


data architecture SKILL
data infrastructure SKILL
data pipeline SKILL
azure data factory SKILL
azure data factory SKILL
azure data lake SKILL
analytics SKILL
data storage SKILL
sql SKILL
database SKILL
analytics SKILL
data mart SKILL
azure data factory SKILL
sql SKILL
azure data lake SKILL
data pipeline SKILL
ci cd SKILL
azure databricks SKILL
analytics SKILL
power bi SKILL
sql SKILL
database SKILL
azure data factory SKILL
data store SKILL
c SKILL
computer science SKILL
azure data factory SKILL
data modeling SKILL
database management SKILL
sql azure SKILL
sql SKILL
azure data lake SKILL
data warehousing SKILL
problem solving SKILL
project management SKILL


In [146]:
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.util import filter_spans

nlp = spacy.load("en_core_web_sm")  # Load the model
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")  # Create the matcher object

# Assuming 'skills_list' is a list of skills, and 'job_descriptions' is a list containing job descriptions

# Add patterns to the matcher. Patterns are made by converting each skill string into a Doc object
patterns = [nlp.make_doc(skill) for skill in skills_list]
matcher.add("Skills", patterns)

# Process the job description to create a Spacy Doc
doc = nlp(course_description[4])

# Match the patterns to the doc
matches = matcher(doc)

# Create Span objects for the matched sequences
spans = [Span(doc, start, end, label="SKILL") for match_id, start, end in matches]

# Filter the spans to remove overlaps
filtered_spans = filter_spans(spans)

# Now you can create new entities in the doc using the filtered spans
doc.ents = filtered_spans  # Overwrite or append to doc.ents with the non-overlapping skill entities

# Print the entities in the document
for ent in doc.ents:
    print(ent.text, ent.label_)


artificial intelligence SKILL
deep learning SKILL
artificial intelligence SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
algorithm SKILL
deep learning SKILL
deep learning SKILL
algorithm SKILL
artificial neural network SKILL
convolutional neural network SKILL
boltzmann machine SKILL
autoencoders SKILL
autoencoders SKILL
deep learning SKILL
deep learning SKILL
dataset SKILL
algorithm SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
deep learning SKILL
algorithm SKILL
deep learning SKILL
deep learning SKILL
analytics SKILL
dataset SKILL
dataset SKILL
artificial neural network SKILL
probability SKILL
deep learning SKILL
deep learning SKILL
convolutional neural network SKILL
deep learning SKILL
deep

In [147]:
filtered_spans

[artificial intelligence,
 deep learning,
 artificial intelligence,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 algorithm,
 deep learning,
 deep learning,
 algorithm,
 artificial neural network,
 convolutional neural network,
 boltzmann machine,
 autoencoders,
 autoencoders,
 deep learning,
 deep learning,
 dataset,
 algorithm,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 deep learning,
 algorithm,
 deep learning,
 deep learning,
 analytics,
 dataset,
 dataset,
 artificial neural network,
 probability,
 deep learning,
 deep learning,
 convolutional neural network,
 deep learning,
 deep learning,
 deep learning,
 artificial intelligence,
 deep learning,
 deep learning method,
 deep learning,
 deep learning,
 deep learning,
 dataset,
 amazon,
 dataset,
 dataset,
 dataset,
 d

In [158]:
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.util import filter_spans

nlp = spacy.load("en_core_web_sm")  # Load the model
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")  # Create the matcher object



# Convert the skills list to Doc objects and add them as patterns to the matcher
patterns = [nlp.make_doc(skill) for skill in skills_list]
matcher.add("Skills", patterns)

TRAIN_DATA = []

# Loop through the job descriptions and create training data
for job_description in job_descriptions:
    # Process the job description to create a Spacy Doc
    doc = nlp(job_description)
    # Match the patterns to the doc
    matches = matcher(doc)
    # Create Span objects for the matched sequences
    spans = [Span(doc, start, end, label="SKILLS") for match_id, start, end in matches]
    # Filter the spans to remove overlaps
    #print(spans)
    filtered_spans = filter_spans(spans)
    #print(spans)
    entities = [(span.start_char, span.end_char, span) for span in filtered_spans]
    TRAIN_DATA.append((job_description, {"entities": entities}))

#print(TRAIN_DATA)


In [193]:
TRAIN_DATA

[('description data engineer responsible creation ongoing maintenance internal external data pipeline service includes timely resolution data issue customer ad hoc data request assigned data architect leadership team collaborate product owner operation process owner identify opportunity define business requirement assist designing implementing solution assist reporting critical process system responsibility life cycle project analysis development delivery solution modify existing process improve operational efficiency entire data processing team perform data mining aggregation combining multiple datasets develop streamlined efficient solution internal external party perform data validation analysis ensure accuracy quality data handle data conversion data cleansing report generation scheduled data delivery standard ability deliver data feature based operation leadership requirement requirement required experience year data engineering experience relative analytical engineering experienc

In [174]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        print(ent)

(20, 31, data center)
(133, 144, data center)
(355, 366, data center)
(387, 398, data center)
(912, 923, data center)
(1062, 1073, data center)
(1517, 1528, data center)
(2138, 2149, data center)
(355, 364, analytics)
(481, 497, data engineering)
(498, 514, cloud technology)
(520, 529, analytics)
(547, 556, analytics)
(748, 761, data modeling)
(762, 765, sql)
(807, 808, c)
(947, 965, data visualization)
(1016, 1034, data visualization)
(1074, 1090, data engineering)
(1111, 1127, data engineering)
(1128, 1144, cloud technology)
(1192, 1211, data infrastructure)
(1230, 1239, analytics)
(1288, 1297, analytics)
(1353, 1373, business requirement)
(1383, 1392, analytics)
(1423, 1432, analytics)
(1443, 1462, predictive modeling)
(1480, 1500, statistical analysis)
(2184, 2199, problem solving)
(2226, 2229, sql)
(2230, 2247, data manipulation)
(2296, 2297, c)
(2381, 2399, data visualization)
(2400, 2416, data engineering)
(2417, 2433, cloud technology)
(2439, 2448, analytics)
(2466, 2475, analy

In [192]:
#train the model 
starting_fresh = False
# Load a pre-existing spaCy model
nlp = spacy.load('en_core_web_sm')  # for example
import random
from spacy.util import minibatch
# Get the Named Entity Recognizer component in the pipeline
ner = nlp.get_pipe('ner')
from spacy.training import Example
from pathlib import Path

# Add new entity labels to 'ner'
for _, annotations in TRAIN_DATA:
    for start,end,label in annotations.get('entities'):
        print(label)
        ner.add_label(str(label))


# Disable other pipes during training
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']


# Begin training
with nlp.disable_pipes(*unaffected_pipes):
    if starting_fresh:
        nlp.begin_training()

    for itn in range(100):
        random.shuffle(TRAIN_DATA)
        losses = {}

        # Batch up the examples using spaCy's minibatch
        for batch in minibatch(TRAIN_DATA, size=2):
            examples = []
            for text, annotations in batch:
                # Create a Spacy Doc from the text
                doc = nlp.make_doc(text)
                # Create an Example using the annotations
                example = Example.from_dict(doc, annotations)
                examples.append(example)

            # Update the model
            nlp.update(
                examples,
                drop=0.5,  # Dropout - make it harder to memorize data
                losses=losses
            )
        print(losses)






Object `Path` not found.
visualization
azure devops
data pipeline
data pipeline
data ingestion
sql
data pipeline
computer science
data engineering
sql
cloud technology
airflow
big data
data acquisition
java
data extraction
analytics
dataflow
data extraction
analytics
business intelligence
data management
data storage
data warehousing
test data
computer science
database design
business intelligence
sql
sql
sql
database tuning
data modeling
ci cd
sql
database design
c
sql
sql
c
c
data engineering
computer science
java
sql
data structure
batch processing
apache kafka
ci cd
data pipeline
sql
data management
sql
sql
business requirement
database
sql
sql
business intelligence
database
database design
problem solving
sql
sql
database architecture
business intelligence
agile methodology
c
asp net
sql
sql
database architecture
sql
data engineering
data system
data mapping
database
data integration
data quality
predictive modeling
data management
data analysis
sql
data modeling
database
sql
post

In [195]:
from pathlib import Path
output_dir = Path('/Users/nyzy/nitzmali/job_transition_pathway/models/skills_tag_spacy_nlp_model')
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to /Users/nyzy/nitzmali/job_transition_pathway/models/skills_tag_spacy_nlp_model


In [197]:
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
from spacy.scorer import Scorer
from pathlib import Path
def train_test_val_split(data, test_size=0.2, val_size=0.25, random_state=42):
    # Calculate actual validation set size of the remaining data after test split
    val_size_adjusted = val_size / (1 - test_size)
    # Split off test set from available data
    train_val_data, test_data = train_test_split(data, test_size=test_size, random_state=random_state)
    # Split remaining data into training and validation sets
    train_data, val_data = train_test_split(train_val_data, test_size=val_size_adjusted, random_state=random_state)
    return train_data, val_data, test_data




In [208]:
# Load the model you want to evaluate
nlp = spacy.load('/Users/nyzy/nitzmali/job_transition_pathway/models/skills_tag_spacy_nlp_model')  # replace with your model name

# Split your TRAIN_DATA into train, validate and test sets
train_data, val_data, test_data = train_test_val_split(TRAIN_DATA, test_size=0.2, val_size=0.25)

# Convert the validation data to spaCy's Example format
examples = []
for text, annots in TRAIN_DATA:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annots)
    examples.append(example)


# Use the Scorer to score the examples
scorer = Scorer(nlp)
scores = scorer.score(examples)


precision = scores['ents_p']
recall = scores['ents_r']
f_score = scores['ents_f']

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F-score: {f_score}")

Precision: 0.0
Recall: 0.0
F-score: 0.0


In [218]:
# Convert the validation data to spaCy's Example format
examples = []
for text, annots in val_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annots)
    examples.append(example)


# Use the Scorer to score the examples
scorer = Scorer(nlp)
scores = scorer.score(examples)
scores['ents_per_type']

{'c': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'analytics': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'visualization': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data engineering': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'sql': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'database': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data pipeline': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'airflow': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data infrastructure': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'application data': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data lake': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'computer science': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data curation': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'mysql': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'big data': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'nosql': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'artificial intelligence': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data warehousing': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'apache beam': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'sql azure': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'database development': {'p': 0.0, 'r':

In [310]:
examples[0]
from spacy import displacy
nlp = spacy.load('/Users/nyzy/nitzmali/job_transition_pathway/models/skills_tag_spacy_nlp_model')  # replace with your model name

# Assume 'nlp' is your loaded NLP model
for text, annots in val_data:  # Let's use val_data as an example
    doc = nlp(text)  # Process the text to predict entities
    print("Predictions by model:")
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)

    # Now print the correct data for comparison
    print("\nCorrect labels:")
    for start, end, label in annots['entities']:
        print(text[start:end], start, end, label)

    # You can use displacy here as well if you prefer visual comparison
    displacy.render(doc, style="ent", jupyter=True)

    # Adding a separation for readability between different examples
    print("\n" + "-" * 50 + "\n")


SyntaxError: EOL while scanning string literal (3146176248.py, line 21)

In [215]:
# Access and print scores
scores['ents_per_type']

{'power bi': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data conversion': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'sql': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data processing': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'bigquery': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'business requirement': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'java': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data validation': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data mining': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data pipeline': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data cleansing': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data engineering': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data visualization': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'relational database management system': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'business intelligence': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'http': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data governance': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data architecture': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'business analysis': {'p': 0.0, 'r': 0.0, 'f': 0.0},
 'data science': {'p': 0.0

In [222]:
# Assuming 'val_data' is available and properly formatted as per spaCy requirements


# Checking entities on new or unseen text
for text, annots in val_data:
    doc_gold_text = nlp.make_doc(text)  # the document text
    gold = Example.from_dict(doc_gold_text, annots)  # the gold-standard Example object

    # Using the model to predict the outcome on the same text
    doc_pred = nlp(text)  # the predicted document
    pred_ents = [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc_pred.ents]  # predicted entities

    # Comparing predicted entities with the gold standard
    print("Predicted entities:", pred_ents)
    print("Gold entities:", gold.reference.ents)
    
    # Check if there are any predicted entities that match the gold standard
    for pred in doc_pred.ents:
        for gold_span in gold.reference.ents:
            if pred.text == gold_span.text and pred.label_ == gold_span.label_ and pred.start_char == gold_span.start_char and pred.end_char == gold_span.end_char:
                print(f"Matched entity: {pred.text} ({pred.label_})")
    
    # If the loop above never finds a match, you may want to check your training and annotation process.


Predicted entities: [('database', 845, 853, 'database'), ('analytics', 984, 993, 'analytics'), ('data pipeline', 1430, 1443, 'data pipeline'), ('airflow', 1457, 1464, 'airflow'), ('application data', 1480, 1496, 'application data'), ('data engineering', 1684, 1700, 'data engineering'), ('data engineering', 1733, 1749, 'data engineering'), ('data pipeline', 1818, 1831, 'data pipeline'), ('data pipeline', 1949, 1962, 'data pipeline'), ('sql', 2008, 2011, 'sql'), ('sql', 2025, 2028, 'sql'), ('data infrastructure', 2077, 2096, 'data infrastructure'), ('analytics', 2097, 2106, 'analytics'), ('visualization', 2107, 2120, 'visualization'), ('c', 2747, 2748, 'c')]
Gold entities: (database, analytics, data pipeline, airflow, application data, data engineering, data engineering, data pipeline, data pipeline, sql, sql, data infrastructure, analytics, visualization, c)
Matched entity: database (database)
Matched entity: analytics (analytics)
Matched entity: data pipeline (data pipeline)
Matched en

In [224]:
for text, annots in val_data:
    doc_gold_text = nlp.make_doc(text)  # the document text
    gold = Example.from_dict(doc_gold_text, annots)  # the gold-standard Example object

    # Using the model to predict the outcome on the same text
    doc_pred = nlp(text)  # the predicted document

    # Print predicted entities with their span info
    pred_ents = [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc_pred.ents]
    print("Predicted entities with positions and labels:", pred_ents)

    # Print the gold standard entity annotations for comparison
    gold_ents = [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in gold.reference.ents]
    print("Gold standard entities with positions and labels:", gold_ents)

    # Find matched entities
    matched_entities = []
    for pred in doc_pred.ents:
        match_found = any(
            pred.text == gold_ent.text and pred.label_ == gold_ent.label_ and pred.start_char == gold_ent.start_char and pred.end_char == gold_ent.end_char
            for gold_ent in gold.reference.ents
        )
        if match_found:
            matched_entities.append((pred.text, pred.start_char, pred.end_char, pred.label_))

    print("Matched entities:", matched_entities)


Predicted entities with positions and labels: [('database', 845, 853, 'database'), ('analytics', 984, 993, 'analytics'), ('data pipeline', 1430, 1443, 'data pipeline'), ('airflow', 1457, 1464, 'airflow'), ('application data', 1480, 1496, 'application data'), ('data engineering', 1684, 1700, 'data engineering'), ('data engineering', 1733, 1749, 'data engineering'), ('data pipeline', 1818, 1831, 'data pipeline'), ('data pipeline', 1949, 1962, 'data pipeline'), ('sql', 2008, 2011, 'sql'), ('sql', 2025, 2028, 'sql'), ('data infrastructure', 2077, 2096, 'data infrastructure'), ('analytics', 2097, 2106, 'analytics'), ('visualization', 2107, 2120, 'visualization'), ('c', 2747, 2748, 'c')]
Gold standard entities with positions and labels: [('database', 845, 853, 'database'), ('analytics', 984, 993, 'analytics'), ('data pipeline', 1430, 1443, 'data pipeline'), ('airflow', 1457, 1464, 'airflow'), ('application data', 1480, 1496, 'application data'), ('data engineering', 1684, 1700, 'data enginee

In [294]:
# Convert the validation data to spaCy's Example format
examples = []
for text, annots in train_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annots)
    examples.append(example)


# Use the Scorer to score the examples
scorer = Scorer(nlp)
scores = scorer.score(examples)


precision = scores['ents_p']
recall = scores['ents_r']
f_score = scores['ents_f']

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F-score: {f_score}")



Precision: 0.0
Recall: 0.0
F-score: 0.0


In [302]:
doc = nlp(job_description)
for ent in doc.ents:
    print(ent.text, ent.label_)

data infrastructure data infrastructure
data system data system
data pipeline data pipeline
data system data system
ad hoc analysis ad hoc analysis
database database
data extraction data extraction
computer science computer science
data quality data quality
data warehousing data warehousing
data modeling data modeling
analytics analytics
database database
postgresql postgresql
sql sql
data infrastructure data infrastructure
apache airflow apache airflow


In [308]:

scorer = Scorer(nlp)
example = []
for input_, annotations in train_data:
    pred = nlp(input_)
    #print(pred,annotations)
    temp = Example.from_dict(pred, annotations)
    example.append(temp)
scores = scorer.score(example)
scores

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'tag_acc': None,
 'sents_p': None,
 'sents_r': None,
 'sents_f': None,
 'dep_uas': None,
 'dep_las': None,
 'dep_las_per_type': None,
 'pos_acc': None,
 'morph_acc': None,
 'morph_micro_p': None,
 'morph_micro_r': None,
 'morph_micro_f': None,
 'morph_per_feat': None,
 'lemma_acc': None,
 'ents_p': 0.9941451990632318,
 'ents_r': 0.991822429906542,
 'ents_f': 0.9929824561403509,
 'ents_per_type': {'analytics': {'p': 1.0,
   'r': 0.9803921568627451,
   'f': 0.99009900990099},
  'data engineering': {'p': 1.0, 'r': 0.975, 'f': 0.9873417721518987},
  'cloud technology': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'data modeling': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'sql': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'c': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'data visualization': {'p': 0.9375, 'r': 1.0, 'f': 0.967741935483871},
  'data infrastructure': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'business requirement': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'pre

In [306]:
dict.fromkeys(annotations)

{'entities': None}

In [307]:
annotations

{'entities': [(404, 420, data engineering),
  (444, 460, data acquisition),
  (465, 472, dataset),
  (771, 791, cloud infrastructure),
  (931, 944, data pipeline),
  (962, 982, business requirement),
  (1514, 1529, data governance),
  (1598, 1614, data engineering),
  (1638, 1654, data engineering),
  (1659, 1666, dataset),
  (1706, 1710, java),
  (1835, 1841, devops),
  (1856, 1861, ci cd),
  (1937, 1953, computer science)]}

In [300]:
job_description

'department engineering location cary north carolina united state product epic online service company epic game requisition id r make epic core epic success talented passionate people epic pride creating collaborative welcoming creative environment building award winning game crafting engine technology enables visually stunning interactive experience innovating epic mean team continually strives right community user constantly innovating raise bar engine game development ecosec ecosec team provides safer experience epic user work multiple product service improve technology craft transparent policy player user positive experience platform responsible designing building maintaining data infrastructure ensure reliability efficiency data system ecosystem security team role include building maintaining data pipeline transform load data product managing aws infrastructure machine learning platform additionally work engineer product manager data scientist design implement robust scalable data

In [ ]:
scorer.score

In [269]:
examples

[{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-data pipeline', 'L-data pipeline', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-business requirement', 'L-business requirement', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-data processing', 'L-data processing', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [289]:
from spacy.training import Example
from spacy.scorer import Scorer
import spacy



# Convert the test data to spaCy's Example format
examples = []
for text, annots in tes:
    doc = nlp.make_doc(text)  # Create a Doc object from the text
    example = Example.from_dict(doc, annots)  # Create an Example object from the Doc and annotations
    examples.append(example)

scorer = Scorer()

for input_, annot in examples:
    doc_gold_text = nlp.make_doc(input_)
    gold = GoldParse(doc_gold_text, entities=annot['entities'])
    pred_value = nlp(input_)
    scorer.score(pred_value, gold)



TypeError: cannot unpack non-iterable spacy.training.example.Example object

In [290]:
examples

[{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-data pipeline', 'L-data pipeline', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-business requirement', 'L-business requirement', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-data processing', 'L-data processing', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [291]:
scorer.score(examples)

{'token_acc': 1.0,
 'token_p': 1.0,
 'token_r': 1.0,
 'token_f': 1.0,
 'sents_p': None,
 'sents_r': None,
 'sents_f': None,
 'tag_acc': None,
 'pos_acc': None,
 'morph_acc': None,
 'morph_micro_p': None,
 'morph_micro_r': None,
 'morph_micro_f': None,
 'morph_per_feat': None,
 'dep_uas': None,
 'dep_las': None,
 'dep_las_per_type': None,
 'ents_p': 0.0,
 'ents_r': 0.0,
 'ents_f': 0.0,
 'ents_per_type': {'business requirement': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'data pipeline': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'data processing': {'p': 0.0, 'r': 0.0, 'f': 0.0}},
 'cats_score': 0.0,
 'cats_score_desc': 'macro F',
 'cats_micro_p': 0.0,
 'cats_micro_r': 0.0,
 'cats_micro_f': 0.0,
 'cats_macro_p': 0.0,
 'cats_macro_r': 0.0,
 'cats_macro_f': 0.0,
 'cats_macro_auc': 0.0,
 'cats_f_per_type': {},
 'cats_auc_per_type': {}}

In [281]:
examples

[{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-data pipeline', 'L-data pipeline', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-business requirement', 'L-business requirement', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-data processing', 'L-data processing', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [276]:
pred_value

NameError: name 'pred_value' is not defined

In [245]:
from spacy.training import Example
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()  # Scorer does not take model as argument
    for example in examples:
        # Score the example
        scorer.score(example)
    return scorer.scores

# Prepare examples as a list of Example objects
examples = [Example.from_dict(nlp.make_doc(text), annot) for text, annot in train_data[0]]

# Evaluate the model with the examples
scores = evaluate(nlp, examples)

# Print out the scores
print(f"Precision: {scores['ents_p']:.3f}")
print(f"Recall: {scores['ents_r']:.3f}")
print(f"F-score: {scores['ents_f']:.3f}")


ValueError: too many values to unpack (expected 2)

In [257]:
train_data[0]

('direct hire opportunity client hybrid role located minneapolis mn candidate able work sponsorship seeking highly motivated detail oriented data engineer join dynamic team ideal candidate responsible developing maintaining data solution drive business objective forward passion working data technical background desire contribute project leverage advanced analytics role excellent opportunity meaningful impact work collaboratively team create innovative data driven solution focus data engineering cloud technology data analytics product advanced analytics ready career level difference data driven world encourage apply responsibility data solution development design develop implement data solution address business need leveraging understanding data modeling sql proficiency coding skill language python c javascript continuous improvement identify seize opportunity enhance existing data solution optimizing performance scalability efficiency data visualization collaborate cross functional tea

In [251]:
len(train_data[0])

2

In [244]:
train_data[0]

('direct hire opportunity client hybrid role located minneapolis mn candidate able work sponsorship seeking highly motivated detail oriented data engineer join dynamic team ideal candidate responsible developing maintaining data solution drive business objective forward passion working data technical background desire contribute project leverage advanced analytics role excellent opportunity meaningful impact work collaboratively team create innovative data driven solution focus data engineering cloud technology data analytics product advanced analytics ready career level difference data driven world encourage apply responsibility data solution development design develop implement data solution address business need leveraging understanding data modeling sql proficiency coding skill language python c javascript continuous improvement identify seize opportunity enhance existing data solution optimizing performance scalability efficiency data visualization collaborate cross functional tea

In [93]:
skills_list

['visualization',
 'b testing',
 'ansi c',
 'api gateway',
 'api management',
 'apl programming language',
 'asp net',
 'asp net core',
 'asp net core mvc',
 'asp net extension ajax',
 'asp net fundamental',
 'asp net identity',
 'asp net mvc',
 'asp net mvc framework',
 'asp net razor',
 'asp net web api',
 'awk programming language',
 'aws amplify',
 'aws app mesh',
 'aws appsync',
 'aws auto scaling',
 'aws backup',
 'aws batch',
 'aws cli command line interface',
 'aws cloud development kit cdk',
 'aws cloudformation',
 'aws cloudhsm',
 'aws cloudtrail',
 'aws codebuild',
 'aws codecommit',
 'aws codedeploy',
 'aws codepipeline',
 'aws cost management',
 'aws directory service',
 'aws elastic beanstalk',
 'aws elastic mapreduce emr',
 'aws fargate',
 'aws glue',
 'aws identity access management iam',
 'aws inferentia',
 'aws internet thing iot',
 'aws key management service km',
 'aws kinesis',
 'aws lambda',
 'aws opsworks',
 'aws outpost',
 'aws sdk',
 'aws sagemaker',
 'aws serv

In [96]:
job_descriptions[0]

'randstad technology data center site engineer located wilmington de relocation assistance available global financial leader exciting data center opportunity physical hand essential worker creative schedule enables extra day compared normal day week family free weekly covid testing opportunity growth advancement unique workload day thing day world class data center seeking experienced data center technician long term project role located wilmington de area basic responsibility include limited rack stack enterprise class technology network storage enterprise server running tracing dressing testing copper fiber cable patch panel device familiarity patch panel troubleshooting diagnostics hardware connectivity commission decommission hardware able perform component replacement dimms hard drive power supply adhere change control process required skill previous experience working complex high availability data center environment providing layer hardware installation ongoing maintenance probl

'randstad technology data center site engineer located wilmington de relocation assistance available global financial leader exciting data center opportunity physical hand essential worker creative schedule enables extra day compared normal day week family free weekly covid testing opportunity growth advancement unique workload day thing day world class data center seeking experienced data center technician long term project role located wilmington de area basic responsibility include limited rack stack enterprise class technology network storage enterprise server running tracing dressing testing copper fiber cable patch panel device familiarity patch panel troubleshooting diagnostics hardware connectivity commission decommission hardware able perform component replacement dimms hard drive power supply adhere change control process required skill previous experience working complex high availability data center environment providing layer hardware installation ongoing maintenance probl

In [66]:
df_j

,Unnamed: 0,company,viewJobLink,companyBrandingAttributes,companyOverviewLink,companyRating,companyReviewCount,displayTitle,employerAssistEnabled,employerResponsive,expired,extractedSalary,featuredCompanyAttributes,featuredEmployer,featuredEmployerCandidate,feedId,formattedLocation,formattedRelativeTime,highVolumeHiringModel,hiringEventJob,indeedApplyEnabled,indeedApplyable,isJobVisited,isMobileThirdPartyApplyable,isNoResumeJob,isSubsidiaryJob,jobCardRequirementsModel,jobLocationCity,jobLocationState,jobTypes,locationCount,newJob,normTitle,openInterviewsInterviewsOnTheSpot,openInterviewsJob,openInterviewsOffersOnTheSpot,openInterviewsPhoneJob,overrideIndeedApplyText,pubDate,rankingScoresModel,remoteLocation,resumeMatch,salarySnippet,screenerQuestionsURL,showAttainabilityBadge,showCommutePromo,showEarlyApply,showJobType,showRelativeDate,showSponsoredLabel,showStrongerAppliedLabel,smartFillEnabled,smbD2iEnabled,snippet,sponsored,taxoAttributes,taxoAttributesDisplayLimit,taxoLogAttributes,taxonomyAttributes,thirdPartyApplyUrl,title,translatedAttributes,translatedCmiJobTags,truncatedCompany,urgentlyHiring,vjFeaturedEmployerCandidate,jobDescription
0,0,Randstad Digital,/viewjob?jk=7a1a3b7d5ebc2fee&from=vjs&tk=1hec8...,{'headerImageUrl': 'https://d2q79iu7y748jz.clo...,https://www.indeed.com/cmp/Randstad?campaignid...,3.7,18140.0,Data Center Engineer - DE,False,False,False,"{'max': 32, 'min': 26, 'type': 'hourly'}",{},False,False,50461,"Bear, DE",30+ days ago,{'highVolumeHiring': False},False,True,True,False,False,False,False,"{'additionalRequirementsCount': -2, 'requireme...",Bear,DE,[],1,False,Data Engineer,False,False,False,False,True,1675749600000,"{'bid': 16528, 'eApply': 0.008056486, 'eQualif...",False,False,"{'currency': 'USD', 'salaryTextFormatted': Fal...",iq://de2bf8bcc5b264d786f9?v=1,False,False,False,False,True,False,False,False,False,Randstad Technologies Data Center Site Enginee...,True,[],3,[],[{'attributes': [{'label': 'Employee discount'...,http://www.indeed.com//applystart?jk=7a1a3b7d5...,data center engineer de,[],[],Randstad Digital,True,False,Randstad Technologies\nData Center Site Engine...
1,1,Emergent Software,/viewjob?jk=9e15b51f14153cff&from=vjs&tk=1hec8...,{'headerImageUrl': 'https://d2q79iu7y748jz.clo...,https://www.indeed.com/cmp/Emergent-Software?c...,4.8,15.0,Data Engineer,False,False,False,"{'max': 125000, 'min': 100000, 'type': 'yearly'}",{},False,False,50461,"Minneapolis-Saint Paul, MN",17 days ago,{'highVolumeHiring': False},False,False,False,False,False,False,False,"{'additionalRequirementsCount': 0, 'requiremen...",Minneapolis-Saint Paul,MN,[],2,False,Data Engineer,False,False,False,False,True,1697518800000,"{'bid': 4000, 'eApply': 0.030367099, 'eQualifi...",False,False,"{'currency': 'USD', 'salaryTextFormatted': Fal...",NaN,False,False,False,False,True,False,False,False,False,_* This is a direct-hire opportunity with our ...,True,[],3,[],[{'attributes': [{'label': 'Bonus opportunitie...,http://www.indeed.com//applystart?jk=9e15b51f1...,data engineer,[],[],Emergent Software,False,False,** This is a direct-hire opportunity with our ...
2,2,"CGI Group, Inc.",/viewjob?jk=fabba5776ab608a2&from=vjs&tk=1hec8...,{'headerImageUrl': 'https://d2q79iu7y748jz.clo...,https://www.indeed.com/cmp/CGI?campaignid=mobv...,3.6,3570.0,Technical Lead - AWS Data Engineer (Python) - ...,False,False,False,NaN,{},False,False,111415,"Columbia, SC 29228",2 days ago,{'highVolumeHiring': False},False,False,False,False,True,False,False,"{'additionalRequirementsCount': 0, 'requiremen...",Columbia,SC,[],1,True,Technical Lead,False,False,False,False,True,1698814800000,"{'bid': 7530, 'eApply': 0.002060637, 'eQualifi...",False,False,"{'currency': 'USD', 'salaryTextFormatted': False}",NaN,False,False,False,False,True,False,False,False,False,Technical Lead - AWS Data Engineer (Python) - ...,True,[],3,[],"[{'attributes': [], 'label': 'dynamic-attribut...",http://www.indeed.com//applystart?jk=fabba5776...,technical lead aws data enginee

In [45]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r